# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("Tema3")\
    .getOrCreate()

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("Datasets/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [6]:
FlightsDelayedDF = cleanFlightsDF.groupBy("Origin","Dest")\
                                 .agg(
                                        F.mean("ArrDelay").alias("avgDelay"),\
                                        F.count("*").alias("count"))\
                                 .sort(F.col("avgDelay").desc())
FlightsDelayedDF.show()

+------+----+------------------+-----+
|Origin|Dest|          avgDelay|count|
+------+----+------------------+-----+
|   RDM| MFR|            1347.0|    2|
|   MDT| HPN|             798.0|    1|
|   ORD| GTF|             212.0|    1|
|   ICT| DAY|             210.0|    1|
|   ELM| ATL|             169.0|    2|
|   DSM| PIA|             168.0|    1|
|   ERI| ITH|             160.0|    1|
|   YNG| PIE|             141.0|    1|
|   CMH| HOU|             120.0|    1|
|   HRL| DAL|             111.0|    1|
|   PPG| HNL|109.85714285714286|   35|
|   HNL| PPG|105.85714285714286|   35|
|   PIE| YNG|             104.0|    1|
|   AVP| SFB|              93.0|    1|
|   ACY| MSY| 87.45454545454545|   11|
|   CPR| LAS|              85.0|    1|
|   LAS| CPR|              82.0|    1|
|   TTN| BNA|              76.5|   10|
|   MSP| PVD|              74.0|    1|
|   TUL| OKC|              69.0|    1|
+------+----+------------------+-----+
only showing top 20 rows



In [7]:
AggFlights = cleanFlightsDF.groupBy(F.col("Origin"), F.col("Dest")).agg(
                F.round(F.mean(F.col("ArrDelay")),1).alias("Average_delay"),
                F.count("*").alias("Number_flights"))\
            .orderBy(F.col("Average_delay"), ascending = False)
    
AggFlights.show()

+------+----+-------------+--------------+
|Origin|Dest|Average_delay|Number_flights|
+------+----+-------------+--------------+
|   RDM| MFR|       1347.0|             2|
|   MDT| HPN|        798.0|             1|
|   ORD| GTF|        212.0|             1|
|   ICT| DAY|        210.0|             1|
|   ELM| ATL|        169.0|             2|
|   DSM| PIA|        168.0|             1|
|   ERI| ITH|        160.0|             1|
|   YNG| PIE|        141.0|             1|
|   CMH| HOU|        120.0|             1|
|   HRL| DAL|        111.0|             1|
|   PPG| HNL|        109.9|            35|
|   HNL| PPG|        105.9|            35|
|   PIE| YNG|        104.0|             1|
|   AVP| SFB|         93.0|             1|
|   ACY| MSY|         87.5|            11|
|   CPR| LAS|         85.0|             1|
|   LAS| CPR|         82.0|             1|
|   TTN| BNA|         76.5|            10|
|   MSP| PVD|         74.0|             1|
|   TUL| OKC|         69.0|             1|
+------+---

In [8]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin", "Dest").agg(
                                        F.round(F.mean("ArrDelay"), 2).alias("avgDelay"),
                                        F.round(F.count("*"), 2).alias("cuantos")
                                    ).sort("avgDelay", ascending = False)

sortedAvgDelaysDF.show()

+------+----+--------+-------+
|Origin|Dest|avgDelay|cuantos|
+------+----+--------+-------+
|   RDM| MFR|  1347.0|      2|
|   MDT| HPN|   798.0|      1|
|   ORD| GTF|   212.0|      1|
|   ICT| DAY|   210.0|      1|
|   ELM| ATL|   169.0|      2|
|   DSM| PIA|   168.0|      1|
|   ERI| ITH|   160.0|      1|
|   YNG| PIE|   141.0|      1|
|   CMH| HOU|   120.0|      1|
|   HRL| DAL|   111.0|      1|
|   PPG| HNL|  109.86|     35|
|   HNL| PPG|  105.86|     35|
|   PIE| YNG|   104.0|      1|
|   AVP| SFB|    93.0|      1|
|   ACY| MSY|   87.45|     11|
|   CPR| LAS|    85.0|      1|
|   LAS| CPR|    82.0|      1|
|   TTN| BNA|    76.5|     10|
|   MSP| PVD|    74.0|      1|
|   TUL| OKC|    69.0|      1|
+------+----+--------+-------+
only showing top 20 rows



In [10]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin", "Dest", "ArrDelayCat").agg(
                                        F.count("*").alias("cuantos"))\
                                    .where("Origin = 'MCI' and Dest = 'MKE'").show()

+------+----+-----------+-------+
|Origin|Dest|ArrDelayCat|cuantos|
+------+----+-----------+-------+
|   MCI| MKE|       Huge|      8|
|   MCI| MKE|       None|    172|
|   MCI| MKE|     Slight|     18|
+------+----+-----------+-------+



### Vamos a desplegar ArrDelayCat

In [11]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin", "Dest").pivot("ArrDelayCat").agg(
                                        F.count("*").alias("cuantos")
)
sortedAvgDelaysDF.show()

+------+----+----+----+------+
|Origin|Dest|Huge|None|Slight|
+------+----+----+----+------+
|   MCI| MKE|   8| 172|    18|
|   TPA| ACY|   4| 112|     4|
|   PBI| DCA|  37| 393|    49|
|   DSM| EWR|  10|  94|    14|
|   MDW| MEM|  22| 172|    42|
|   ORD| PDX|  23| 528|    85|
|   SHD| LWB|   2|  25|  null|
|   SMF| BUR|  61| 720|   124|
|   STS| PHX|   9| 105|    14|
|   MCI| IAH|  38| 487|    54|
|   FSD| ATL|   9|  83|     9|
|   PHL| MCO| 162|1291|   273|
|   ATL| GSP|  47|1080|   106|
|   SJC| LIH|   1|  83|     5|
|   DSM| MCO|   1|  30|    10|
|   IAD| ILM|   2|  34|     8|
|   PBG| PGD|   1|  19|     6|
|   LBB| DEN|  20| 184|    20|
|   SNA| PHX|  57| 967|   256|
|   PIE| AVP|null|   1|  null|
+------+----+----+----+------+
only showing top 20 rows



In [12]:
cleanFlightsDF.groupBy("Origin")\
              .pivot("ArrDelayCat")\
              .count().show()

+------+----+-----+------+
|Origin|Huge| None|Slight|
+------+----+-----+------+
|   BGM|  32|  223|    27|
|   PSE|  26|  194|    27|
|   INL|  21|  172|    14|
|   PPG|   4|   21|    10|
|   MSY|1298|15297|  2262|
|   GEG| 207| 5836|   596|
|   SNA| 705|12038|  1464|
|   BUR| 637| 7025|  1253|
|   GRB| 173| 1441|   180|
|   GTF|  59|  731|    86|
|   IFP|null|   31|    14|
|   IDA|  32|  699|    61|
|   LWB|   5|   43|    10|
|   GRR| 667| 5068|   863|
|   PVU|  14|  133|    22|
|   JLN|  27|  197|    53|
|   EUG| 157| 2360|   261|
|   PSG|  12|  211|    17|
|   GSO| 528| 4011|   623|
|   MYR| 262| 2335|   290|
+------+----+-----+------+
only showing top 20 rows



<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [13]:
pivot = cleanFlightsDF.groupBy("Origin")\
                      .pivot("ArrDelayCat").agg(
                          F.count("*").alias("Conteo"),
                          F.max("ArrDelay").alias("maxArrDelay"))\
                        .sort("Origin")
pivot.show()

+------+-----------+----------------+-----------+----------------+-------------+------------------+
|Origin|Huge_Conteo|Huge_maxArrDelay|None_Conteo|None_maxArrDelay|Slight_Conteo|Slight_maxArrDelay|
+------+-----------+----------------+-----------+----------------+-------------+------------------+
|   ABE|        193|           674.0|       1230|            14.0|          239|              59.0|
|   ABI|         56|           397.0|        497|            14.0|           76|              59.0|
|   ABQ|        457|          1650.0|       7184|            14.0|          947|              59.0|
|   ABR|         22|           584.0|        198|            13.0|           20|              40.0|
|   ABY|         38|           641.0|        239|            14.0|           57|              50.0|
|   ACT|         40|           500.0|        386|            14.0|           42|              59.0|
|   ACV|         40|           900.0|        291|            14.0|           33|              59.0|


In [14]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest").agg(
    F.mean("ArrDelay").alias("avgArrDelay")
)

In [15]:
cleanFlightsDF.groupBy("Origin", "Dest").pivot("ArrDelayCat").agg(
        F.count("*").alias("count"),
        F.mean("ArrDelay").alias("avgDelay")
    ).show()

+------+----+----------+------------------+----------+-------------------+------------+------------------+
|Origin|Dest|Huge_count|     Huge_avgDelay|None_count|      None_avgDelay|Slight_count|   Slight_avgDelay|
+------+----+----------+------------------+----------+-------------------+------------+------------------+
|   MCI| MKE|         8|              89.5|       172| -8.325581395348838|          18|31.166666666666668|
|   TPA| ACY|         4|              79.0|       112| -9.857142857142858|           4|              27.0|
|   PBI| DCA|        37|127.38461538461539|       393|-13.061068702290076|          49| 33.42857142857143|
|   DSM| EWR|        10|             121.8|        94|-12.085106382978724|          14|29.714285714285715|
|   MDW| MEM|        22|102.33333333333333|       172| -9.087209302325581|          42|33.095238095238095|
|   ORD| PDX|        23| 102.0952380952381|       528| -8.910984848484848|          85|29.141176470588235|
|   SHD| LWB|         2|             

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

In [16]:
avgDelaysDF = flightsDF.groupBy("DayOfWeek").agg(
    F.mean("ArrDelay").alias("avgArrDelay"),
    F.mean("DepDelay").alias("avgDepDelay")
).sort("DayOfWeek")
avgDelaysDF.show()

+---------+-------------------+------------------+
|DayOfWeek|        avgArrDelay|       avgDepDelay|
+---------+-------------------+------------------+
|        1|  5.391113068725289|10.430177708665964|
|        2| 2.8412409647873806| 8.246502522185226|
|        3| 3.0525338339576717|  8.47071347600168|
|        4| 2.7390527404801026|  8.35856546210902|
|        5|  5.027363815430113|10.220785437977693|
|        6|-0.5748593305876211| 6.278199328016013|
|        7| 3.2344449424598207| 9.142161259888235|
+---------+-------------------+------------------+



### Cuando utilizo pivot con una sola función de agregación, el alias es ignorado

In [17]:
flightsPd = flightsDF.groupby("Origin").pivot("DayOfWeek").agg(
    F.mean("ArrDelay").alias("MeanArrDelay")
).sort("Origin").toPandas()

flightsPd

,Origin,1,2,3,4,5,6,7
0,ABE,14.953307,13.893519,15.276786,5.924370,18.183333,6.940299,10.027149
1,ABI,10.650000,16.364706,-0.547619,0.376344,4.641304,16.492063,4.606383
2,ABQ,1.442438,0.363406,1.736409,3.111111,-0.176892,-1.655721,-0.252019
3,ABR,10.571429,12.468750,6.294118,16.441176,8.529412,0.125000,9.117647
4,ABY,14.207547,3.978723,12.804348,10.645833,9.040816,29.562500,27.234043
...,...,...,...,...,...,...,...,...
351,XNA,9.592857,9.375580,6.838462,1.854015,3.877941,-0.977208,5.438861
352,YAK,-9.451613,-13.000000,-10.727273,2.848485,-9.470588,-12.088235,-10.090909
353,YKM,0.338462,-0.490909,5.666667,1.359375,6.269841,5.941176,1.603175
354,YNG,9.000000,NaN,NaN,141.000000,NaN,NaN,NaN


### Pero cuando utilizo varias funciones de agregación, entonces sí es necesario

### Spark concatena con "_" cada valor de la variable pivotada con mis alias de las func de agregación

In [18]:
flightsDF.groupby("Origin").pivot("DayOfWeek").agg(
    F.mean("ArrDelay").alias("MeanArrDelay"),
    F.min("ArrDelay").alias("MinArrDelay")
    ).sort("Origin").toPandas()

,Origin,1_MeanArrDelay,1_MinArrDelay,2_MeanArrDelay,2_MinArrDelay,3_MeanArrDelay,3_MinArrDelay,4_MeanArrDelay,4_MinArrDelay,5_MeanArrDelay,5_MinArrDelay,6_MeanArrDelay,6_MinArrDelay,7_MeanArrDelay,7_MinArrDelay
0,ABE,14.953307,-41.0,13.893519,-32.0,15.276786,-34.0,5.924370,-42.0,18.183333,-37.0,6.940299,-33.0,10.027149,-36.0
1,ABI,10.650000,-28.0,16.364706,-27.0,-0.547619,-23.0,0.376344,-22.0,4.641304,-27.0,16.492063,-23.0,4.606383,-28.0
2,ABQ,1.442438,-67.0,0.363406,-61.0,1.736409,-67.0,3.111111,-61.0,-0.176892,-52.0,-1.655721,-63.0,-0.252019,-59.0
3,ABR,10.571429,-27.0,12.468750,-33.0,6.294118,-31.0,16.441176,-28.0,8.529412,-29.0,0.125000,-31.0,9.117647,-30.0
4,ABY,14.207547,-34.0,3.978723,-34.0,12.804348,-20.0,10.645833,-25.0,9.040816,-21.0,29.562500,-17.0,27.234043,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,XNA,9.592857,-39.0,9.375580,-46.0,6.838462,-49.0,1.854015,-43.0,3.877941,-50.0,-0.977208,-51.0,5.438861,-41.0
352,YAK,-9.451613,-45.0,-13.000000,-41.0,-10.727273,-46.0,2.848485,-34.0,-9.470588,-36.0,-12.088235,-50.0,-10.090909,-41.0
353,YKM,0.338462,-16.0,-0.490909,-21.0,5.666667,-28.0,1.359375,-18.0,6.269841,-18.0,5.941176,-21.0,1.603175,-30.0
354,YNG,9.000000,9.0,NaN,NaN,NaN,NaN,141.000000,141.0,NaN,NaN,NaN,NaN,NaN,NaN


### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

In [22]:
tempDF = cleanFlightsDF.join(averageDelayOriginDestDF, 
                               on = (cleanFlightsDF["Origin"] == averageDelayOriginDestDF["Origin"]) & 
                                    (cleanFlightsDF["Dest"] == averageDelayOriginDestDF["Dest"])
                            )\
                         .select(cleanFlightsDF["Origin"], cleanFlightsDF["Dest"], "DayOfWeek", "ArrDelay", "avgArrDelay")

tempDF.show()

AnalysisException: Column Origin#20, Dest#22 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.

In [23]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest")\
                                         .agg(F.mean("ArrDelay").alias("avgDelay"))

joinedDF = cleanFlightsDF.join(averageDelayOriginDestDF, 
                               on = ["Origin", "Dest"], 
                               how = "left_outer")\
                         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgDelay")

joinedDF.show()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|
|   ATL| GSP|        3|    47.0|-1.734910277324633|
|   ATL| GSP|        4|    null|-1.734910277324633|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|
|   ATL| GSP|        4|    38.0|-1.734910277324633|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        1|    14.0|-1.734910277324633|
|   ATL| GSP|        1|    -3.0|-1.734910277324633|
|   ATL| GSP|        2|   -22.0|-1.734910277324633|
|   ATL| GSP

In [24]:
from pyspark.sql import Window
w = Window().partitionBy("Origin", "Dest")
joinedWindowDF = cleanFlightsDF.withColumn("avgArrDelay", F.mean("ArrDelay").over(w))\
                               .select("Origin","Dest","DayOfWeek","ArrDelay","avgArrDelay")\
                               .withColumn("belowAverage", F.col("ArrDelay") < F.col("avgArrDelay"))
joinedWindowDF.show()

+------+----+---------+--------+------------------+------------+
|Origin|Dest|DayOfWeek|ArrDelay|       avgArrDelay|belowAverage|
+------+----+---------+--------+------------------+------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    47.0|-1.734910277324633|       false|
|   ATL| GSP|        4|    null|-1.734910277324633|        null|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        4|    38.0|-1.734910277324633|       false|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|        true|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|        true|
|   ATL| GSP|        1|    10.0|-1.734910277324633|       false|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|        true|
|   ATL| GSP|        1|  

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

In [25]:
from pyspark.sql import Window
w = Window().partitionBy("Origin", "Dest")

withAvgDF = cleanFlightsDF.withColumn("avgDelay", F.mean("ArrDelay").over(w))
withAvgDF.where("Origin = 'ATL' and Dest = 'GSP'")\
         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgDelay").show()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|
|   ATL| GSP|        3|    47.0|-1.734910277324633|
|   ATL| GSP|        4|    null|-1.734910277324633|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|
|   ATL| GSP|        4|    38.0|-1.734910277324633|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        1|    14.0|-1.734910277324633|
|   ATL| GSP|        1|    -3.0|-1.734910277324633|
|   ATL| GSP|        2|   -22.0|-1.734910277324633|
|   ATL| GSP

### La solución de Cayetano al bonus

In [26]:
bonusDF = withAvgDF.withColumn("belowAverage", 
                              F.when(F.col("ArrDelay") < F.col("avgDelay"), True).otherwise(False))
bonusDF.select("Origin", "Dest", "ArrDelay", "avgDelay", "belowAverage").show()

+------+----+--------+------------------+------------+
|Origin|Dest|ArrDelay|          avgDelay|belowAverage|
+------+----+--------+------------------+------------+
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|    47.0|-1.734910277324633|       false|
|   ATL| GSP|    null|-1.734910277324633|       false|
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    38.0|-1.734910277324633|       false|
|   ATL| GSP|   -13.0|-1.734910277324633|        true|
|   ATL| GSP|   -15.0|-1.734910277324633|        true|
|   ATL| GSP|    10.0|-1.734910277324633|       false|
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|   -14.0|-1.734910277324633|        true|
|   ATL| GSP|    10.0|-1.734910277324633|       false|
|   ATL| GSP|    14.0|-1.734910277324633|       false|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| G

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

### Inciso: cómo crear un DF de Spark a partir de un dataframe de Pandas

In [27]:
import pandas as pd
airports_pd = pd.DataFrame({"IATA": ["JFK", "LIT", "SEA"],
                            "Year": [1948, 1931, 1949]})
airportsFromPandasDF = spark.createDataFrame(airports_pd)
airportsFromPandasDF.show()

+----+----+
|IATA|Year|
+----+----+
| JFK|1948|
| LIT|1931|
| SEA|1949|
+----+----+



In [28]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [29]:
# cleanFlights["Origin"] == airportsDF["IATA"]
# F.col("Origin") == airportsDF("IATA")

joinedFlightsDF = cleanFlightsDF.join(airportsDF, 
                                      on = cleanFlightsDF.Origin == airportsDF.IATA, 
                                      how = "left_outer")

# PREGUNTA: mostrar algunas filas donde FullName no sea null
# equivalente: filter("FullName is not null")
joinedFlightsDF.filter(~(F.col("FullName").isNull()))\
               .select("Origin", "Dest", "IATA", "FullName", "Year")\
               .show(10, truncate = False)

+------+----+----+------------------------------------+----+
|Origin|Dest|IATA|FullName                            |Year|
+------+----+----+------------------------------------+----+
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
+------+----+----+------------------------------------+----+
only showing top 10 rows



## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0
+------+----+--------+------------------+
|Origin|Dest|Distance|            DistKM|
+------+----+--------+------------------+
|   CLE| JFK|   425.0|             680.0|
|   MCO| PSE|  1179.0|            1886.4|
|   FLL| DTW|  1127.0|            1803.2|
|   LAX| MTJ|   666.0|1065.6000000000001|
|   AZA| MLI|  1288.0|            2060.8|
+------+----+--------+------------------+
only showing top 5 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [31]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daysOfWeek[dayInteger]

print(dayOfWeekToString(2))

# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = F.udf(dayOfWeekToString, StringType())

# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()

Tuesday
+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   BQK| ATL|        4|       Thursday|
|   CVG| PHL|        3|      Wednesday|
|   DTW| DFW|        5|         Friday|
|   SEA| JFK|        2|        Tuesday|
|   JAX| JFK|        2|        Tuesday|
|   RDU| BOS|        3|      Wednesday|
|   SEA| BOS|        3|      Wednesday|
|   AUS| FLL|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   SLC| BOS|        6|       Saturday|
|   BOS| HOU|        6|       Saturday|
|   BDL| MCO|        7|         Sunday|
|   SJU| TPA|        7|         Sunday|
|   PGD| TYS|        6|       Saturday|
|   PIE| CVG|        6|       Saturday|
|   ABE| SFB|        7|         Sunday|
|   LAS| BIS|        7|         Sunday|
|   ROC| PGD|        1|         Monday|
|   EWR| CVG|        1|         Monday|
|   CVG| SFB|        1|         Monday|
+------+----+---------+---------------+
only showing top 20 rows

